In [1]:
import os
import numpy as np
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredPDFLoader,UnstructuredFileLoader,OnlinePDFLoader
#from langchain_text_splitters import RecursiveCharacterTextSplitter
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import faiss
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter

In [2]:
embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 96}
)

C:\Users\drief\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
# Define the loader for PDF and text files
loader = DirectoryLoader('documents/', glob="*.txt", loader_cls=TextLoader, loader_kwargs={'encoding': 'UTF-8'})
# Load documents
documents = loader.load()

text_splitter = CharacterTextSplitter()
docs = text_splitter.split_documents(documents)
embeddings = embed_model
db = FAISS.from_documents(docs, embeddings)
print(db.index.ntotal)

C:\Users\drief\AppData\Roaming\Python\Python39\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


385


In [4]:
query = "qui est le coordinateur de la MST intelligence artificielle et sciences des données ?"
docs = db.similarity_search_with_score(query)

In [5]:
print(docs[0])#.page_content)

(Document(page_content="Filière: MST : Intelligence Artificielle et Sciences de Données\n\nObjectif: Cette formation d'excellence offre de solides connaissances en conception de systèmes d'intelligence artificielle et mathématiques appliquées afin de couvrir l'ensemble des problématiques de traitement et d'analyse des données massives que rencontre les entreprises. Elle met l'accent sur l'articulation entre apprentissage automatique, gestion et fouille de grandes masses de données, paradigmes du Big Data, représentation des connaissances, le traitement des données et sur les méthodologies récemment développées.\n\nCoordinateur: Pr.EZZIYYANI MOSTAFA\nEmail: mezziyyani@uae.ac.ma\n\nDébouchés: \n\nProgramme:\nSemestre 1:\n  1. THÉORIES ET SYSTÈMES DE RAISONNEMENTS INTELLIGENTS\n  2. MATHÉMATIQUES POUR ANALYSE DE DONNÉES\n  3. PROGRAMMATION AVANCÉE\n  4. BASES DE DONNÉES AVANCÉES\n  5. MACHINE LEARNING 1\n  6. ANGLAIS ET TECHNIQUE D’EXPRESSION\n\nSemestre 2:\n  1. INFRASTRUCTURE ET ARCHI

In [6]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model,language = "fr",model_kwargs={"device": "cuda"},temperature=0.5)

In [7]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Vous êtes un assistant académique de la faculté des sciences et techniques de tanger modèle linguistique IA. Votre tâche consiste à générer deux
    versions différentes de la question utilisateur donnée pour récupérer les documents pertinents d'une
    base de données vectorielle. En générant plusieurs perspectives sur la question de l'utilisateur,
    votre objectif est d'aider l'utilisateur à surmonter certaines des limitations de la recherche par similarité basée sur la distance.
    Utilisez les informations du deux premiers documents retournés pour formuler ces questions alternatives.
    Fournissez ces questions alternatives séparées par des nouvelles lignes.
    Question originale : {question}""",
)


In [8]:
retriever = MultiQueryRetriever.from_llm(db.as_retriever(), llm,prompt=QUERY_PROMPT)
#retriever = db.as_retriever()
# RAG prompt
template = """Répondez à la question en français sur la base du contexte suivant :
{context}
Question : {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
#print(chain.invoke("la faculté des sciences et technques de tanger?"))


In [9]:
%%time
print(chain.invoke("la date de création de la faculté des sciences et technques de tanger?"))

 The provided text does not contain the exact creation date of the Faculty of Sciences and Techniques (FST) of Tangier. However, it mentions that the Institut Confucius for the teaching of Chinese language and culture was created three years ago on October 21, 2019, which gives a hint about the recent establishment of FST or related institutions. To find the exact creation date of FST, you should consult their official website or other reliable sources.
CPU times: total: 531 ms
Wall time: 1min 5s


In [10]:
%%time
print(chain.invoke(" MST intelligence artificielle et sciences des données ?"))

 The Faculty of Sciences and Technologies (FSTT) offers a Master's degree in Science and Technology (MST), which includes Artificial Intelligence (AI) and Data Science as specializations. However, the exact title for this specific program might not be explicitly mentioned in the provided text. You may want to check their official website or contact the Future Leaders Club for more detailed information about the MST program related to AI and Data Science.
CPU times: total: 500 ms
Wall time: 1min


In [11]:
%%time
print(chain.invoke("la faculté des sciences et technques de tanger?"))

 La Faculté des Sciences et Techniques (FST) de Tanger est un établissement relevant de l'Université Abdelmalek Essaâdi, fondé en 1995. Elle offre une variété de programmes académiques et de recherche dans les domaines des sciences et de l'ingénierie. La FSTT propose une panoplie de formation riches et diversifiées répondant aux divers besoins du monde socio-économique (12 filières LST, 13 filières MST, 4 filières Ingénieurs). Elle est structurée en plusieurs départements spécialisés, tels que : Génie Électrique, Génie Mécanique, Génie Informatique, Génie Chimique, Mathématiques, Physique, Sciences de la Vie, Sciences de la Terre. La FSTT se distingue par une recherche active, soutenue par 7 laboratoires, 11 équipes de recherche, et des centres comme le Centre de Développement et d’Innovation (CDI) et le Centre de Fabrication Additive (CFA). Les thématiques de recherche incluent l'agro-alimentaire, la biotechnologie, et l'environnement. La FSTT maintient des liens étroits avec le milie

In [12]:
%%time
print(chain.invoke("intelligence artificielle et sciences des données ?"))

 Les documents suivants semblent être liés à l'intelligence artificielle (IA) et aux sciences des données (SD):

1. Document avec le numéro 3 dans la liste, intitulé "Ingénierie des données". Ce document n'a pas de contenu spécifique sur la page, mais étant donné que l'ingénierie des données est une branche de l'IA et des SD, il pourrait être pertinent.
2. Document avec le numéro 4 dans la liste, intitulé "PFE". Bien qu'il n'y ait pas de contenu spécifique sur la page, les PFE (projets fin d'études) sont souvent utilisés pour développer des compétences en IA et SD.
3. Document avec le numéro 3 dans la liste, intitulé "Formation continue". Ce document parle des programmes de formation continue destinés aux professionnels qui souhaitent acquérir des nouvelles compétences ou approfondir leurs connaissances dans leur domaine d'activité. Les domaines couverts incluent les sciences de l'ingénierie, de l'informatique et d'autres disciplines qui peuvent être liées à l'IA et aux SD.
4. Document

In [13]:
%%time
print(chain.invoke("les concours de FSTT"))

 Les concours suivants sont actuellement ouverts à la FSTT (Faculté des Sciences et Technologies de Tanger) :

1. Avis de concours pour le recrutement de deux (02) Professeurs – Assistants réservé aux fonctionnaires titulaires d’un Doctorat. Date : juin 25, 2021. Pour plus de détails, veuillez consulter [ce lien](https://fstt.ac.ma/Portail2023/avis-de-concours-pour-le-recrutement-de-deux-02-professeurs-assistants-au-profit-des-fonctionnaires-titulaires-d-un-doctorat/)

2. Avis de concours de recrutement de trois (03) Maîtres de Conférences. Date : octobre 13, 2023. Pour plus de détails, veuillez consulter [ce lien](https://fstt.ac.ma/Portail2023/avis-de-concours-de-recrutement-de-trois-03-maitres-de-conferences/)

Pour obtenir des informations supplémentaires, vous pouvez consulter les documents suivants :

* [Avis de concours pour le recrutement de deux (02) Professeurs – Assistants réservé aux fonctionnaires titulaires d’un Doctorat](documents\\Avis_de_concours_pour_le_recrutement_de

In [14]:
%%time
print(chain.invoke("la date de création de la faculté des sciences et technques de tanger?"))

 La question n'a pas été trouvée dans les documents fournis. Les documents fournis contiennent des informations sur la coopération et les partenariats, des programmes d'études, des opportunités de recherche, des compétitions, des événements et des formations, mais pas de date de création de la faculté des sciences et technologie de Tanger.
CPU times: total: 375 ms
Wall time: 54.5 s


In [15]:
%%time
print(chain.invoke("C'est qui monsieur DIANI ?"))

 Monsieur Diani ne semble pas être mentionné dans les documents que vous avez fournis. Il pourrait être lié à un document manquant ou non disponible dans la liste de documents que vous m'avez transmise. Vérifiez-vous s'il existe un autre document qui pourrait vous aider à répondre à votre question.
CPU times: total: 250 ms
Wall time: 1min 10s


In [16]:
%%time
print(chain.invoke("C'est qui monsieur DIANI mustafa ?"))

 Based on the provided data, there is no information about a person named "DIANI Mustafa" in the list of documents. The roles or positions mentioned are related to academic departments or announcements, not individual people.
CPU times: total: 297 ms
Wall time: 43.4 s


In [17]:
%%time
print(chain.invoke("C'est qui monsieur DIANI ?"))

 Il n'y a pas de référence à un Monsieur DIANI dans les documents fournis.
CPU times: total: 250 ms
Wall time: 41.5 s


In [18]:
%%time
print(chain.invoke("C'est qui monsieur DIANI mustafa ?"))

 Il n'y a pas de référence à un "monsieur DIANI Mustafa" dans les documents fournis. Ainsi, il est impossible de déterminer l'identité de cette personne.
CPU times: total: 688 ms
Wall time: 55.5 s


In [19]:
%%time
print(chain.invoke("Le capitale de Maroc?"))

 La réponse à cette question est "Rabat". Rabat est la capitale du Royaume du Maroc. Les documents fournis ne fournissent pas d'informations sur la capitale, mais ils traitent tous des projets et événements liés au pays.
CPU times: total: 406 ms
Wall time: 46.2 s


In [21]:
%%time
print(chain.invoke("Bonsoir"))


KeyboardInterrupt

